In [1]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

/workspaces/scrape-linkedIn/linkedin_scraper/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
try:
    if load_dotenv("dotfiles/.env"):
        genai.configure(api_key=os.getenv('GEMINI_API_KEY'))
        models = genai.list_models()
        print("If your API key is valid. All available models will be listed below...")
        for model in models:
            print(model.display_name)
except Exception as e:
    print("API KEY is invalid!!")

If your API key is valid. All available models will be listed below...
PaLM 2 Chat (Legacy)
PaLM 2 (Legacy)
Embedding Gecko
Gemini 1.0 Pro
Gemini 1.0 Pro 001 (Tuning)
Gemini 1.0 Pro Latest
Gemini 1.0 Pro Vision
Gemini 1.5 Flash
Gemini 1.5 Pro
Gemini 1.0 Pro
Gemini 1.0 Pro Vision
Embedding 001
Text Embedding 004
Model that performs Attributed Question Answering.


In [34]:
# Create the model
# See https://ai.google.dev/api/python/google/generativeai/GenerativeModel
generation_config = {
  "temperature": 1.0,
  "top_p": 1,
  "top_k": 0,
  "max_output_tokens": 14000,
  "response_mime_type": "text/plain",
}
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
]

system_instruction = '''You are an agent that takes in a given piece of text and then creates a dictionary accordingly. \
                        You are given a person's professional experience where you have to convert it into key, value pairs in the following manner where the keys signify: \
                        Company, Role, Location, Start_Date, End_Date, Duration, Type_of_work(on-site, hybrid, remote), Description, Skills_Used. \
                        Note that you have to only use Strings enclosed in triple quotes for the value of Description key, the rest using normal double quotes; \ 
                        but include a list of skills for Skills_Used.
                        Enclose all of the generated dictionaries into a super dictionary with key_name as Experience wherein each individual experience is take as \
                        key, value pairs: key: numbers(starting from zero), values being each professional experience.'''

In [35]:
model = genai.GenerativeModel(
  model_name="gemini-1.0-pro",
  safety_settings=safety_settings,
  generation_config=generation_config,
  # system_instruction=system_instruction
)


In [36]:
experience = '''
Jaws Shark
Full-time · 2 yrs
Another temporary experience
May 2023 to Present · 1 yr 1 mo
Hyderabad, Telangana, India · Hybrid
Temporary, not permanent.
Fishing · Sailing
Temporary Experience (For a scraper that I am building)
Jun 2022 to May 2023 · 1 yr
Visakhapatnam, Andhra Pradesh, India · On-site
Nothing much to say other than that this is temporary
Team Leadership · Fishing
Machine Learning Summer Intern
CloudKarya, Inc · Internship
Apr 2023 to Jun 2023 · 3 mos
Visakhapatnam, Andhra Pradesh, India · On-site
Developed a Deep Learning solution to detect the presence of Retinal Diseases. The website has been deployed on Google Cloud Platform and available for public use: https://dev-ikshana-dw5whmz2hq-ue.a.run.app/getdata Techstack used: - Docker - Docker Compose - Google Cloud Run - Google Cloud Function - Google Cloud Storage Bucket - Google BigQuery - Convolution-Neural-Networks - Tensorflow - Vanilla Framework (HTML, CSS, Javascript)
Team Leadership · docker · Web Development · Google Cloud Platform (GCP) · TensorFlow
RPA Developer
SS&C Blue Prism · Internship
Aug 2022 to Sep 2022 · 2 mos
Remote
I have been a part of the RPA Developer internship offered as part of AICTE's NEAT program. The internship was provided by EduSkills foundation and SS&C Blue Prism University. I learnt about the basics of RPA and Bot development using Blueprism Studio.
Analytical Skills · Blue Prism · Critical Thinking · Robotic Process Automation (RPA)
'''

In [37]:
chat_session = model.start_chat()
initial_response = chat_session.send_message('''
You are an agent that takes in a given piece of text and then creates a dictionary accordingly. \
                        You are given a person's professional experience where you have to convert it into key, value pairs in the following manner where the keys signify: \
                        Company, Role, Location, Start_Date, End_Date, Duration, Type_of_work(on-site, hybrid, remote), Description, Skills_Used. \
                        Note that you have to only use Strings enclosed in triple quotes for the value of Description key, the rest using normal double quotes; \ 
                        but include a list of skills for Skills_Used.
                        Enclose all of the generated dictionaries into a super dictionary with key_name as Experience wherein each individual experience is take as \
                        key, value pairs: key: numbers(starting from zero), values being each professional experience; dont' add anything else. \
                        Generate only the required dictionary, but not as markdown text. That is, do not generate the dictionary as embedded code in Markdown. \
                        Reply only the word 'YES' if you've understood the instruction
''')

In [38]:
dictionary_response = None
if initial_response.text == "YES":
    dictionary_response = chat_session.send_message(experience)

In [39]:
print(dictionary_response.text)

{
 "Experience": {
  "0": {
   "Company": "Jaws Shark",
   "Role": "Full-time",
   "Location": null,
   "Start_Date": "May 2023",
   "End_Date": "Present",
   "Duration": "2 yrs",
   "Type_of_work": null,
   "Description": null,
   "Skills_Used": []
  },
  "1": {
   "Company": "Another temporary experience",
   "Role": null,
   "Location": "Hyderabad, Telangana, India",
   "Start_Date": "May 2023",
   "End_Date": "Present",
   "Duration": "1 yr 1 mo",
   "Type_of_work": "Hybrid",
   "Description": "\"Temporary, not permanent.\"",
   "Skills_Used": [
    "Fishing",
    "Sailing"
   ]
  },
  "2": {
   "Company": "Temporary Experience (For a scraper that I am building)",
   "Role": null,
   "Location": "Visakhapatnam, Andhra Pradesh, India",
   "Start_Date": "Jun 2022",
   "End_Date": "May 2023",
   "Duration": "1 yr",
   "Type_of_work": "On-site",
   "Description": "\"Nothing much to say other than that this is temporary\"",
   "Skills_Used": [
    "Team Leadership",
    "Fishing"
   ]
 

In [32]:
final_response = chat_session.send_message("End chat")

In [33]:
print(final_response.text)

Goodbye!
